## Counter

To implement a counter we can use the `Register` primitive and the `+` operator. How can we make a more general counter of arbitray width?

In [1]:
import magma as m
from mantle import Register

The generator for a counter is parameterized by the number of bits in the counter, `n`.

In [2]:
def DefineCounter(n):
    class _Counter(m.Circuit):
        name = f'Counter{n}'
        IO = ["O", m.Out(m.UInt[n])] + m.ClockInterface()
        @classmethod
        def definition(io):
            reg = Register(n, _type=m.UInt)
            io.O <= reg(reg.O + 1)
    return _Counter

Sinulate the counter.

In [3]:
from magma.simulator import PythonSimulator

Counter4 = DefineCounter(4)
sim = PythonSimulator(Counter4, Counter4.CLK)

sim.evaluate()
val = sim.get_value(Counter4.O)
print(sim.get_value(Counter4.O))
assert val == 0

for i in range(1,11): 
    sim.advance() # toggle clock - now High
    assert i == sim.get_value(Counter4.O)
    print(sim.get_value(Counter4.O))

    sim.advance() # toggle clock - now Low
    assert i == sim.get_value(Counter4.O)

print("Success!")

0
1
2
3
4
5
6
7
8
9
10
Success!


In [4]:
m.compile("build/Counter4", Counter4, output="coreir")
!coreir -i build/Counter4.json -p instancecount

/Users/leonardtruong/miniconda3/lib/python3.7/site-packages/pysmt/walkers/generic.py:43: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  if len(nodetypes) == 1 and isinstance(nodetypes[0], collections.Iterable):
An instance count of all the primitives
Counter4 | instances in current | instances in children | 
  coreir_add__width4 | 1 | 0
  coreir_const__width4 | 1 | 0
  coreir_reg__width4 | 1 | 0

{"top":"global.Counter4",
"namespaces":{
  "global":{
    "modules":{
      "Counter4":{
        "type":["Record",[
          ["O",["Array",4,"Bit"]],
          ["CLK",["Named","coreir.clkIn"]]
        ]],
        "instances":{
          "const_1_4":{
            "genref":"coreir.const",
            "genargs":{"width":["Int",4]},
            "modargs":{"value":[["BitVector",4],"4'h1"]}
          },
          "magma_Bits_4_add_inst0":{
            "genref":"coreir.add",
            "genargs":{